In [45]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [46]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics
import math

In [47]:
df = pd.read_csv('/Users/eminozata/Downloads/TrainAndValid.csv',
                low_memory = False,
                parse_dates= ["saledate"])

In [ ]:
df = df.sort_values(by="saledate")

In [48]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Direkt RMSL yapacağımız için y(hat)'i log olarak alıyoruz**

In [49]:
df["SalePrice"] = np.log(df.SalePrice)

In [50]:
df["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
412693       NaN
412694       NaN
412695       NaN
412696       NaN
412697       NaN
Name: UsageBand, Length: 412698, dtype: object

In [51]:
def train_cats(df):
    for n,c in df.items():
        if pd.api.types.is_string_dtype(c):
            df[n] = c.astype("category").cat.as_ordered()

In [52]:
def apply_cats(df,train):
    for n,c in df.items():
        if train[n].dtype == "category":
            df[n] = pd.Categorical(c,categories = train[n].cat.categories,ordered = True)

In [53]:
train_cats(df)

In [54]:
df["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
412693       NaN
412694       NaN
412695       NaN
412696       NaN
412697       NaN
Name: UsageBand, Length: 412698, dtype: category
Categories (3, object): ['High' < 'Low' < 'Medium']

In [55]:
df["UsageBand"].cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [56]:
df["UsageBand"].cat.set_categories(["High", "Medium", "Low"],ordered = True, inplace = True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:2630: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [57]:
df["UsageBand"].cat.categories

Index(['High', 'Medium', 'Low'], dtype='object')

In [58]:
df["UsageBand"].cat.codes

0         2
1         2
2         0
3         0
4         1
         ..
412693   -1
412694   -1
412695   -1
412696   -1
412697   -1
Length: 412698, dtype: int8

In [59]:
def numericalize (df,col,name):
    if not pd.api.types.is_numeric_dtype(col):
        df[name] = col.cat.codes + 1

In [60]:
numericalize(df,df["UsageBand"], "UsageBand")

In [61]:
df["UsageBand"]

0         3
1         3
2         1
3         1
4         2
         ..
412693    0
412694    0
412695    0
412696    0
412697    0
Name: UsageBand, Length: 412698, dtype: int8

# Datetime

In [62]:
df["saledate"]

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
412693   2012-03-07
412694   2012-01-28
412695   2012-01-28
412696   2012-03-07
412697   2012-01-28
Name: saledate, Length: 412698, dtype: datetime64[ns]

In [63]:
df["saledate"].dt.year

0         2006
1         2004
2         2004
3         2011
4         2009
          ... 
412693    2012
412694    2012
412695    2012
412696    2012
412697    2012
Name: saledate, Length: 412698, dtype: int64

In [64]:
def add_datepart(df,dt_name,drop=True):
    dt_column = df[dt_name]
    column_dtype = dt_column.dtype
    
    attr = ["year", "month", "week", "day", "dayofweek", "dayofyear", "is_month_end", "is_month_start", ]
    
    for a in attr:
        df["Date" + a.capitalize()] = getattr(dt_column.dt, a)
    
    df["Date" + "Elapsed"] = dt_column.astype(np.int64)//10**9
    
    if drop:
        df.drop(dt_name, axis = 1, inplace = True)

In [66]:
add_datepart(df,"saledate")

/var/folders/z7/fn50kh0d24z2j8yr5m5ghjrr0000gn/T/ipykernel_88765/2264312377.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["Date" + a.capitalize()] = getattr(dt_column.dt, a)
/var/folders/z7/fn50kh0d24z2j8yr5m5ghjrr0000gn/T/ipykernel_88765/2264312377.py:10: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  df["Date" + "Elapsed"] = dt_column.astype(np.int64)//10**9


In [68]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Steering_Controls,DateYear,DateMonth,DateWeek,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateElapsed
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,3,521D,...,Conventional,2006,11,46,16,3,320,False,False,1163635200
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,3,950FII,...,Conventional,2004,3,13,26,4,86,False,False,1080259200
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,1,226,...,NaN,2004,2,9,26,3,57,False,False,1077753600
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,1,PC120-6E,...,NaN,2011,5,20,19,3,139,False,False,1305763200
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,2,S175,...,NaN,2009,7,30,23,3,204,False,False,1248307200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412693,6333344,9.210340,1919201,21435,149,2.0,2005,NaN,0,30NX,...,NaN,2012,3,10,7,2,67,False,False,1331078400
412694,6333345,9.259131,1882122,21436,149,2.0,2005,NaN,0,30NX2,...,NaN,2012,1,4,28,5,28,False,False,1327708800
412695,6333347,9.433484,1944213,21435,149,2.0,2005,NaN,0,30NX,...,NaN,2012,1,4,28,5,28,False,False,1327708800
412696,6333348,9.210340,1794518,21435,149,2.0,2006,NaN,0,30NX,...,NaN,2012,3,10,7,2,67,False,False,1331078400


# Feather Format

In [69]:
import os

In [73]:
os.makedirs("tmp",exist_ok=True)
df.to_feather("tmp/bulldozers_1")